In [2]:
import psi4
import numpy as np
import resp

In [3]:
psi4.core.set_output_file('output.dat', False)
psi4.set_memory('2 GB')
psi4.set_num_threads(2)


In [23]:
molecule = psi4.geometry("""
 C   1.45051389  -0.06628932   0.00000000
 H   1.75521613  -0.62865986  -0.87500146
 H   1.75521613  -0.62865986   0.87500146
 H   1.92173244   0.90485897   0.00000000
 C  -0.04233122   0.09849378   0.00000000
 O  -0.67064817  -1.07620915   0.00000000
 H  -1.60837259  -0.91016601   0.00000000
 O  -0.62675864   1.13160510   0.00000000
""")


In [24]:
# 計算方法と基底関数の設定
psi4.set_options({
    'basis': '6-31g*',
    'scf_type': 'df',
    'e_convergence': 1e-8,
    'd_convergence': 1e-8
})

# SCF計算の実行
opt_energy, opt_wfn, history = psi4.optimize('scf', return_wfn=True, return_history=True)


Optimizer: Optimization complete!


In [25]:
mol = opt_wfn.molecule()
options = {'VDW_SCALE_FACTORS'  : [1.4, 1.6, 1.8, 2.0],
           'VDW_POINT_DENSITY'  : 1.0,
           'RESP_A'             : 0.0005,
           'RESP_B'             : 0.1,
           }

# Call for first stage fit
charges1 = resp.resp([mol], options)
print('Electrostatic Potential Charges')
print(charges1[0])
print('Restrained Electrostatic Potential Charges')
print(charges1[1])

Electrostatic Potential Charges
[-0.47447237  0.15052981  0.15052981  0.13074298  0.88860939 -0.67612035
  0.45167334 -0.62149261]
Restrained Electrostatic Potential Charges
[-0.29503690  0.10712959  0.10712959  0.08482782  0.80432770 -0.66146609
  0.45334511 -0.60025682]


In [26]:
# Change the value of the RESP parameter A
options['RESP_A'] = 0.001

# Add constraint for atoms fixed in second stage fit
constraint_charge = []
for i in range(4, 8):
    constraint_charge.append([charges1[1][i], [i+1]])
options['constraint_charge'] = constraint_charge
options['constraint_group'] = [[2, 3, 4]]
options['grid'] = ['grid.dat']
options['esp'] = ['grid_esp.dat']

# Call for second stage fit
charges2 = resp.resp([mol], options)

In [27]:
# Get RESP charges
print("\nStage Two:\n")
print('RESP Charges')
print(charges2[1])


Stage Two:

RESP Charges
[-0.29095164  0.09833391  0.09833391  0.09833391  0.80432770 -0.66146609
  0.45334511 -0.60025682]


In [28]:
print("RESP電荷:")
for i, charge in enumerate(charges2[0]):
    symbol = molecule.symbol(i)
    print(f"{symbol}{i+1}: {charge:.6f}")

total_charge = sum(charges2[0])
print(f"\n電荷の合計: {total_charge:.6f}")

RESP電荷:
C1: -0.307203
H2: 0.103751
H3: 0.103751
H4: 0.103751
C5: 0.804328
O6: -0.661466
H7: 0.453345
O8: -0.600257

電荷の合計: 0.000000
